In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
random.seed(42)

In [3]:
class Task:
    next_id = 0
    # Define parameters for the Poisson distribution
    mean_arrival_rate = 1  # Adjust this value as needed


    def __init__(self,arrival_time):
        self.id = Task.next_id
        self.arrival_time = arrival_time
        self.length = random.choice([100,200,300])
        self.deadline = arrival_time + random.randint(1,5)
        Task.next_id += 1

    def __str__(self):
        return "Task " + str(self.id)

    def __repr__(self):
        return "Task " + str(self.id)

    @classmethod
    def create_tasks(cls, n):
        # Generate arrival times using Poisson distribution
        arrival_times = np.sort(np.random.poisson(Task.mean_arrival_rate, n)).astype(int).tolist()
        tasks = [cls(arrival_times[i]) for i in range(n)]
        return tasks

In [4]:
class VM:
    next_id = 0

    def __init__(self):
        self.id = VM.next_id
        VM.next_id += 1
        self.mips = random.choice([20,30])

    def __str__(self):
        return f"VM {self.id}"

    def __repr__(self):
        return f"VM {self.id}"

    @classmethod
    def create_vms(cls, n):
        vms = [cls() for _ in range(n)]
        return vms

In [5]:
def objective_function(solution):
    vm_completion_times = {vm: 0 for _,vm in solution.items()}
    tasks_on_time = 0
    for task, assigned_vm in solution.items():
        task_length = task.length
        vm = assigned_vm
        # Start time considering VM's completion time or task's arrival time
        start_time = max(vm_completion_times[vm], task.arrival_time)
        completion_time = start_time + task_length/vm.mips

        if task.deadline:
          delay = max(0, completion_time - task.deadline)
          if delay == 0:
            tasks_on_time += 1

        vm_completion_times[vm] = completion_time
    success_ratio = tasks_on_time/len(solution)
    fitness = max(vm_completion_times.values())

    return fitness

In [6]:
def initialize_population(pop_size,tasks,vms):
    population = []
    population_set = set()
    while len(population) < pop_size:
      solution = {task: random.choice(vms) for task in tasks}
      # Convert the solution dictionary to a frozenset to make it hashable
      solution_frozenset = frozenset(solution.items())
      if solution_frozenset not in population_set:
        # This solution is unique, add it to the population
        population.append(solution)
        population_set.add(solution_frozenset)
    return population

In [7]:
n_tasks = 3
n_vms = 2


# Usage example
tasks_list = Task.create_tasks(n_tasks) # Task set
vms_list = VM.create_vms(n_vms) # VM set

In [8]:
    # for j,particle in enumerate(swarm):
    #     print(f"Particle {j}")
    #     for task,vm in particle.items():
    #         print(f"{task}({task.arrival_time}) : {vm}  ,Execution Time  ({task.length}/{vm.mips}) = {task.length/vm.mips}")

In [9]:
for task in tasks_list:
    print(f"Task {task.id} ,Length = {task.length} arrived at {task.arrival_time}")

Task 0 ,Length = 300 arrived at 0
Task 1 ,Length = 100 arrived at 0
Task 2 ,Length = 100 arrived at 3


In [10]:
for vm in vms_list:
    print(f"VM {vm.id} , MIPS = {vm.mips}")

VM 0 , MIPS = 20
VM 1 , MIPS = 20


In [17]:
def pso(tasks,vms,swarm_size,max_iter):

  # let us take
  # fitness is taken as makespan
  # position is taken as vm
  # velocity is taken as mips

  w = 0.7298
  c1 = 1.4962
  c2 = 1.4962


  swarm = initialize_population(swarm_size,tasks,vms)
  particle_personal_best = [objective_function(particle) for particle in swarm]
  # Initialize variables to store the global best fitness and the corresponding particle
  global_best_fitness = float('inf')  # Initialize with a high value
  global_best_particle = None

  makespan_history = []  # To store makespan values over iterations


  for i in range(max_iter):
    print(f"Starting Iteration {i} ,Now For Each Particle")
    if global_best_particle:
      print(f"Before Iteration {i}, Best Makespan = {global_best_fitness} is of Particle {swarm.index(global_best_particle)}")
      
    for j,particle in enumerate(swarm):
        print(f"Particle {j}")
        for task,vm in particle.items():
            print(f"{task}({task.arrival_time}) : {vm}  ,Execution Time  ({task.length}/{vm.mips}) = {task.length/vm.mips}")
        # Evaluate fitness of the current particle
        current_fitness = objective_function(particle)
        print("Current Fitness = ",current_fitness)
        print("Particle Personal Best = ",particle_personal_best[j])
        print("Global Best Fitness = ",global_best_fitness)
        
        # Update the personal best if needed
        if current_fitness < particle_personal_best[j]:
            print(f"Particle Personal Best Changed to {current_fitness}")
            particle_personal_best[j] = current_fitness

        # Update the global best if needed
        if current_fitness < global_best_fitness:
            print(f"Global Best Changed to {current_fitness}")
            global_best_fitness = current_fitness
            global_best_particle = particle


        # Update particle velocities and positions
        for task,_ in particle.items():
            print("For",task)
            r1 = random.random()
            r2 = random.random()
            print(f"r1 = {r1} ,r2 = {r2}")
            # Get the current VM assigned to the task
            current_vm = particle[task]

            # Update the velocity for the task-VM assignment
            new_velocity = (w * task.length/current_vm.mips + c1 * r1 * (particle_personal_best[j] - current_fitness) +
                            c2 * r2 * (global_best_fitness - current_fitness))
            print(f"New Velocity = {w} * {task.length} / {current_vm.mips} + {c1} * {r1} * ({particle_personal_best[j]} - {current_fitness}) + {c2} * {r2} * ({global_best_fitness} - {current_fitness})")
            print("New Velocity = ",new_velocity)
            # Update the particle's assignment based on the new velocity
            # This step decides which VM the task will be assigned to
            if new_velocity > 0:
                # Assign the task to a different VM if the velocity is positive
                possible_vms = [vm for vm in vms if vm != current_vm]
                new_assigned_vm = random.choice(possible_vms)
                particle[task] = new_assigned_vm
                print(f'Since New Velocity Is Positive , New Assigned VM = {new_assigned_vm}')
            else:
                print(f'Since New Velocity Is Negative There is No Change')
            print()
        print()
    print(f"After Iteration {i}, Best Makespan = {global_best_fitness} is of Particle {swarm.index(global_best_particle)}")
    print()
    #     print(f"Evaluating for {len(tasks)} Tasks and {len(vms)} VMs... \n At Iteration {i + 1} - Best Makespan: {global_best_fitness}")
    # clear_output(wait=True)
    # Continue with the next iteration
    makespan_history.append(global_best_fitness)  # Collect makespan values
  return global_best_particle, global_best_fitness, makespan_history

In [18]:
# status is shown for before that iteration

In [22]:
swarm_size = 3
max_iter = 4
output = ""

makespan_histories_pso = []

best_solution, best_makespan, makespan_history = pso(tasks_list, vms_list, swarm_size,max_iter)

# print(f"Best Makespan = {best_makespan}")
# output +="For {} Tasks and {} VMs Best Makespan Is {}\n".format(n_tasks,n_vms,best_makespan)

# print(output)

Starting Iteration 0 ,Now For Each Particle
Particle 0
Task 0(0) : VM 1  ,Execution Time  (300/20) = 15.0
Task 1(0) : VM 1  ,Execution Time  (100/20) = 5.0
Task 2(3) : VM 1  ,Execution Time  (100/20) = 5.0
Current Fitness =  25.0
Particle Personal Best =  25.0
Global Best Fitness =  inf
Global Best Changed to 25.0
For Task 0
r1 = 0.09338720007556267 ,r2 = 0.9519996624524752
New Velocity = 0.7298 * 300 / 20 + 1.4962 * 0.09338720007556267 * (25.0 - 25.0) + 1.4962 * 0.9519996624524752 * (25.0 - 25.0)
New Velocity =  10.947
Since New Velocity Is Positive , New Assigned VM = VM 0

For Task 1
r1 = 0.672795670557167 ,r2 = 0.22464066599728827
New Velocity = 0.7298 * 100 / 20 + 1.4962 * 0.672795670557167 * (25.0 - 25.0) + 1.4962 * 0.22464066599728827 * (25.0 - 25.0)
New Velocity =  3.649
Since New Velocity Is Positive , New Assigned VM = VM 0

For Task 2
r1 = 0.14735396224577268 ,r2 = 0.04621371809275843
New Velocity = 0.7298 * 100 / 20 + 1.4962 * 0.14735396224577268 * (25.0 - 25.0) + 1.4962 * 